# Solar Envelope

In this workshop we will learn how to compute the solar envelop of the building based on a voxelized building envelope.

### 0. Initialization
Importing all necessary libraries and specifying the inputs

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
from ladybug.sunpath import Sunpath

In [2]:
vs = 3
unit = [vs, vs, vs]
envelope_path = os.path.relpath('../data/optional_envelope.obj')
context_path = os.path.relpath('../data/immediate_context.obj')

In [3]:
# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)

True


In [4]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(911.927305702112, 874.9174903521119, 870.154989702112),
 (59.703559999999996, 22.69374465, 17.931243999999992),
 (0.0, 0.0, 1.0)]

In [5]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

In [6]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(912.1326102576459, 873.6227949076458, 870.3602942576458),
 (59.703559999999996, 21.19374465, 17.931243999999992),
 (0.0, 0.0, 1.0)]

In [7]:
# initiate sunpath
sp = Sunpath(longitude=4.3571, latitude=52.0116)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
hoys = []
sun_vectors = []
day_multiples = 60
for d in range(365):
    if d%day_multiples==0:
        for h in range(24):
            i = d*24 + h
            # compute the sun object
            sun = sp.calculate_sun_from_hoy(i)
            # extract the sun vector
            sun_vector = sun.sun_vector.to_array()
            # apparantly, if the Z component of sun vector is positive, 
            # it is under the horizon 
            if sun_vector[2] < 0.0:
                hoys.append(i)
                sun_vectors.append(sun_vector)

print(np.array(sun_vectors).shape)

(80, 3)


In [19]:
# constructing the sun direction from the sun vectors in a numpy array
sun_dirs = -np.array(sun_vectors)
# exract the centroids of the envelope voxels
vox_cens = envelope_lattice.centroids
# next step we need to shoot in all of the sun directions from all of the voxels, todo so, we need repeat the sun direction for the number of voxels to construct the ray_dir (which is the list of all ray directions). We need to repeat the voxels for the 
ray_dir = []
ray_src = []
for v_cen in vox_cens:
    for s_dir in sun_dirs:
        ray_dir.append(s_dir)
        ray_src.append(v_cen)
# converting the list of directions and sources to numpy array
ray_dir = np.array(ray_dir)
ray_src = np.array(ray_src)
"""
# Further info: this is the vectorised version of nested for loops
ray_dir = np.tile(sun_dirs, [len(vox_cens),1])
ray_src = np.tile(vox_cens, [1, len(sun_dirs)]).reshape(-1, 3)
"""
print("number of voxels to shoot rays from :",vox_cens.shape)
print("number of rays per each voxel :",sun_dirs.shape)
print("number of rays to be shooted :",ray_src.shape)

number of voxels to shoot rays from : (1390, 3)
number of rays per each voxel : (80, 3)
number of rays to be shooted : (111200, 3)


In [9]:
# computing the intersections of rays with the context mesh
tri_id, ray_id = context_mesh.ray.intersects_id(ray_origins=ray_src, ray_directions=ray_dir, multiple_hits=False)

In [27]:
# computing what percentage of the time through out the year, each voxel sees the sun

# initializing the hits list full of zeros
hits = [0]*len(ray_dir)
# setting the rays that had an intersection to 1
for id in ray_id:
    hits[id] = 1

sun_count = len(sun_dirs)
vox_count = len(vox_cens)
# initiating the list of ratio
vox_sun_acc = []
# iterate over the voxels
for v_id in range(vox_count):
    # counter for the intersection
    int_count = 0
    # iterate over the sun rays
    for s_id in range(sun_count):
        # computing the ray id from voxel id and sun id
        r_id = v_id * sun_count + s_id

        # summing the intersections
        int_count += hits[r_id]
    
    # computing the percentage of the rays that DID NOT have 
    # an intersection (aka could see the sun)
    sun_access = 1.0 - int_count/sun_count

    # add the ratio to list
    vox_sun_acc.append(sun_access)


"""
# Further info: this is the vectorised version of nested for loops
hits = np.full((len(ray_dir)), 0)
hits[ray_id] = 1
hits = hits.reshape(len(vox_src), -1)
voxel_hits = np.sum(hits, axis=1)
vox_sun_acc = 1.0 - voxel_hits / len(sun_dirs)
"""
print(hits.shape)
print(vox_sun_acc)

[0.95, 0.8875, 0.875, 0.775, 0.775, 0.925, 0.95, 0.9375, 0.9125, 0.925, 0.925, 0.875, 0.8375, 0.9375, 0.9, 0.85, 0.8, 0.9, 0.85, 0.825, 0.725, 0.85, 0.8125, 0.7875, 0.75, 0.9375, 0.9375, 0.95, 0.925, 0.9125, 0.9375, 0.925, 0.9125, 0.8875, 0.875, 0.9375, 0.9125, 0.9125, 0.8625, 0.825, 0.9125, 0.8875, 0.8625, 0.825, 0.75, 0.8625, 0.825, 0.825, 0.7875, 0.7125, 0.9375, 0.9375, 0.95, 0.95, 0.9375, 0.9125, 0.925, 0.95, 0.9375, 0.925, 0.9125, 0.9125, 0.9, 0.8625, 0.9375, 0.9375, 0.925, 0.925, 0.8875, 0.85, 0.75, 0.925, 0.9125, 0.8875, 0.875, 0.825, 0.8125, 0.7125, 0.875, 0.8875, 0.8625, 0.825, 0.775, 0.7875, 0.675, 0.95, 0.95, 0.9375, 0.95, 0.9375, 0.9375, 0.95, 0.925, 0.9125, 0.9375, 0.95, 0.925, 0.925, 0.9125, 0.9125, 0.8875, 0.875, 0.825, 0.9375, 0.9375, 0.9375, 0.9375, 0.8875, 0.875, 0.8625, 0.8125, 0.7125, 0.9125, 0.9375, 0.9, 0.9, 0.8625, 0.8625, 0.8, 0.775, 0.6875, 0.8875, 0.9125, 0.875, 0.8625, 0.825, 0.8125, 0.75, 0.7625, 0.6375, 0.95, 0.9625, 0.9625, 0.95, 0.9375, 0.95, 0.95, 0.95, 

### Credits

In [ ]:
__author__ = "Shervin Azadi and Pirouz Nourian"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Solar Envelope"